In [1]:
from util import preprocess

train_seqs = preprocess.get_sequence("data/newly_tokenized/train_auto_tok.tsv")
dev_seqs   = preprocess.get_sequence("data/newly_tokenized/dev_auto_tok.tsv")
test_seqs   = preprocess.get_sequence("data/newly_tokenized/test_auto_tok.tsv")

# 1. Create features

In [2]:
def current_word(i, tokens):
    return {'w0 word': tokens[i][0]}

def current_pos(i, tokens):
    return {'w0 pos': tokens[i][2]}

def prev_pos(i, tokens):
    if i == 0:
        return {}
    if i == 1:
        return {'w-1 pos': tokens[i-1][2]}
    return {'w-1 pos': tokens[i-1][2], 'w-2 pos': tokens[i-2][2]}

In [5]:
print(current_word(2, train_seqs[0]))
print(current_pos(2, train_seqs[0]))
print(prev_pos(2, train_seqs[0]))

{'w0 word': 'ลั่น'}
{'w0 pos': 'VV'}
{'w-1 pos': 'NN', 'w-2 pos': 'NN'}


In [9]:
features = {}
features.update(current_pos(3, train_seqs[0]))
features.update(current_word(3, train_seqs[0]))
features.update(prev_pos(2, train_seqs[0]))
print(features)

{'w0 pos': 'VV', 'w0 word': 'เร่ง', 'w-1 pos': 'NN', 'w-2 pos': 'NN'}


In [3]:
# Featurize text into feature sequences
def featurize(feature_function_list, tokens):
    feature_dict_seq = []
    for i in range(len(tokens)): # ทุก token
        feature_dict = {}
        for feature_fn in feature_function_list: # ทุก feature function
            feature_dict.update(feature_fn(i, tokens))
        feature_dict_seq.append(feature_dict)
    return feature_dict_seq
    
def labelize(tokens):
    return [e[1] for e in tokens]
    
feature_function_list = [current_word, current_pos, prev_pos]

In [4]:
# X_train, X_dev, X_test
feature_seq_list = [featurize(feature_function_list, e) for e in train_seqs]
dev_feature_seq_list = [featurize(feature_function_list, e) for e in dev_seqs]
test_feature_seq_list = [featurize(feature_function_list, e) for e in test_seqs]

# Y_train, Y_dev
label_seq_list = [labelize(e) for e in train_seqs]
dev_label_seq_list = [labelize(e) for e in dev_seqs]

# Train CRF

In [12]:
import sklearn_crfsuite
crf = sklearn_crfsuite.CRF()
crf.fit(feature_seq_list, label_seq_list)

/home/samsatp/contest3/venv/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(keep_tempfiles=None)

In [13]:
pred = crf.predict(feature_seq_list)

### save model

In [ ]:
import joblib
filename = 'models/crf_3_feat.crfsuite'
joblib.dump(crf, filename)

# Load trained model

In [5]:
import joblib
filename = 'models/crf_3_feat.crfsuite'
loaded_model = joblib.load(filename)

# Inference
train_pred = loaded_model.predict(feature_seq_list)
dev_pred = loaded_model.predict(dev_feature_seq_list)
test_pred = loaded_model.predict(test_feature_seq_list)

In [ ]:
import sklearn_crfsuite.metrics
print(sklearn_crfsuite.metrics.flat_classification_report(label_seq_list, train_pred))

In [14]:
def joinSeqTag(seqs, tags):
    output = []
    for i in range(len(seqs)):
        temp = []
        for token, tag in zip(seqs[i], tags[i]):
            if tag != "O":
                temp.append( (token[0],tag) )

        output.append(temp)
        
    return output

test_output = joinSeqTag(test_seqs, test_pred)

from lst20utils import extract_entities
test_pred_output = [extract_entities(e, post=True)[1] for e in test_output]

# Save result

In [19]:
import json
json.dump(test_pred_output, open('result/predicted_dev_entities.json', encoding='utf8', mode='w'))

# Make prediction on Test set
Inferencing test set needs a bit workaround bc. we need POS and test data from corpus, and mapping from a given test data.

In [29]:
# read test file
with open("data/raw/test_set.txt", "r") as f:
    test_file = f.readlines() 

import re
# tokenize
test_seqs_given = [e.strip("\n") for e in test_file]
test_seqs_given = [re.sub("\|","",e) for e in test_seqs_given]

In [15]:
test_sentences = {}

for i, seq in enumerate(test_seqs):
    tokens = [e[0] for e in seq]
    sentence = ''.join(tokens)
    test_sentences[sentence] = test_pred_output[i]


In [36]:
test_pred_output_re = []
for test_sentence in test_seqs_given:
    test_pred_output_re.append( test_sentences[test_sentence] )

In [38]:
import json
json.dump(test_pred_output_re, open('result/test_pred_output_re.json', encoding='utf8', mode='w'))